### Sentiment Analysis PREDICTION - Version 02

+ TFIDF + VADER + NEGATIVEWORD

+ Multidata review source, including VIB, TCB, CTG, etc.

In [1]:
import json
import pandas as pd
import numpy as np

from tqdm import tqdm
from string import punctuation
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import groupby
import sys

import regex as re
from deep_translator import GoogleTranslator
from collections import Counter
import textacy.preprocessing.normalize as tprep
from textacy.preprocessing.remove import accents
import string

import nltk
from nltk.corpus import opinion_lexicon,wordnet,stopwords
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

import os
from datetime import date
import datetime
import time
import shutil
import os

import enchant
dict_english = enchant.Dict("en_US")

#from TopicModeling import *
import pickle

# Android
from google_play_scraper import Sort, reviews, app, reviews_all

# iOS
from app_store_scraper import AppStore

In [2]:
# included code
import sys
sys.path.append('./mylib/')
import global_materials
from load import *
from sentiment_process import *

stopwords = global_materials.stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nlt

In [13]:
# PARAMETER
GET_SENTIMENT_OF_REVIEW_FROM_DATE = datetime.datetime(2022,9,1)
GET_SENTIMENT_OF_REVIEW_TO_DATE = datetime.datetime(2022,9,30)

In [5]:
today = date.today()
PRED_ROOT = './data/predict'

if not os.path.exists("model"):
    os.mkdir("model")
if not os.path.exists("data"):
    os.mkdir("data")
if not os.path.exists(PRED_ROOT):
    os.mkdir(PRED_ROOT)  
if not os.path.exists(PRED_ROOT+"/translate_"+str(today)):
    os.mkdir(PRED_ROOT+"/translate_"+str(today))

________________________________________________________________________________________________________________________
### Data Input

You can get data input from any sources

This example you get data input from App [Android (GooglePlay) and iOS (Appstore)] and check sentiment of new reviews.

In [6]:

rw_android_src_dict = {'vib1':['com.vn.vib.mobileapp','vi','vn'],
                       'vib2':['com.vib.myvib2','vi','vn'],
                       'tcb':['vn.com.techcombank.bb.app','vi','vn'],
                       'tpb':['com.tpb.mb.gprsandroid','vi','vn'],
                       'bid':['vn.com.bidv.ibank','vi','vn'],
                       'vcb':['com.VCB','vi','vn'],
                       'mb':['com.mbcorp','vi','vn'],
                       'ctg':['com.vietinbank.ipay','vi','vn']
                       }
#----------------------------------------------------------------------------------------------------------
# get Android review
column = ['reviewId','userName','userImage','content','score','thumbsUpCount','reviewCreatedVersion','at','replyContent','repliedAt','bankapp']
df_android_rw = pd.DataFrame(columns = column)

get_review_in = ['vib1','vib2'] #rw_android_src_dict.keys()

for bank_name in get_review_in:
    Andrw_df = get_rwdata_android(rw_android_src_dict, bank_name, save_path=PRED_ROOT)   
    Andrw_df['bankapp'] = bank_name
    df_android_rw = pd.concat([df_android_rw, Andrw_df])
    time.sleep(30)
    
df_android_rw.reset_index(drop=True, inplace=True)
df_android_rw.to_csv(PRED_ROOT+'/1_AndRaw_'+str(today)+'_all.csv', index=False)

print(df_android_rw.shape)
df_android_rw.head(1)

#----------------------------------------------------------------------------------------------------------
df_android_rw['month'] = pd.DatetimeIndex(df_android_rw['at']).month
df_android_rw['year'] = pd.DatetimeIndex(df_android_rw['at']).year
df_android_rw['m_y'] = df_android_rw['month'].astype(str)+df_android_rw['year'].astype(str)

#JUST RAW
df_android_used = df_android_rw[['userName','content','at','m_y','score']]
df_android_used.rename(columns = {'userName':'username','content':'review','at':'date','score':'rating'}, inplace=True)

# Convert timetamps to string
df_android_used['date'] = df_android_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
print(df_android_used.shape)
df_android_used.head(1)

vib1: (4686, 10)
vib2: (1696, 10)
(6382, 11)
(6382, 5)


C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_19320\694203429.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_android_used.rename(columns = {'userName':'username','content':'review','at':'date','score':'rating'}, inplace=True)
C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_19320\694203429.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_android_used['date'] = df_android_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))


,username,review,date,m_y,rating
0,huu bui,Muốn mơ tai khoan vib luôn cho căn cước mô dc dk,2022-09-29 11:12:54,92022,4


In [7]:

rw_ios_src_dict = {'vib1':['MyVIB','vn','949371011'],
                   'vib2':['MyVIB 2.0','vn','1626624790'],
                   'tcb':['Techcombank Mobile','vn','1548623362'],
                   'tpb':['TPBank Mobile','vn','450464147'],
                   'bid':['SmartBanking','vn','1061867449'],
                   'vcb':['Vietcombank','vn','561433133'],
                   'mb':['BIZ MBBANK','vn','1424005630'],
                   'ctg':['VietinBank iPay','vn','689963454']
                  }

#----------------------------------------------------------------------------------------------------------
column = ['title','userName','isEdited','review','date','rating','developerResponse','bankapp']
df_ios_rw = pd.DataFrame(columns = column)

get_review_in = ['vib1','vib2'] #rw_ios_src_dict.keys()

for bankapp in get_review_in:
    iosrw_df = get_rwdata_ios(rw_ios_src_dict, bankapp, save_path=PRED_ROOT)   
    iosrw_df['bankapp'] = bankapp
    df_ios_rw = pd.concat([df_ios_rw, iosrw_df])  
    time.sleep(30)
    
today = date.today()
df_ios_rw.reset_index(drop=True, inplace=True)
df_ios_rw.to_csv(PRED_ROOT+'/1_iOSRaw_'+str(today)+'_all.csv', index=False)

print(df_ios_rw.shape)

#----------------------------------------------------------------------------------------------------------
df_ios_rw['month'] = pd.DatetimeIndex(df_ios_rw['date']).month
df_ios_rw['year'] = pd.DatetimeIndex(df_ios_rw['date']).year
df_ios_rw['m_y'] = df_ios_rw['month'].astype(str)+df_ios_rw['year'].astype(str)

#JUST RAW
df_ios_used = df_ios_rw[['userName','review','date','m_y','rating']]
df_ios_used.rename(columns = {'userName':'username'}, inplace=True)

# Convert timetamps to string
df_ios_used['date'] = df_ios_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))



2022-10-01 14:40:31,351 [INFO] Base - Initialised: AppStore('vn', 'myvib', 949371011)
2022-10-01 14:40:31,353 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/vn/app/myvib/id949371011
2022-10-01 14:40:37,235 [INFO] Base - [id:949371011] Fetched 60 reviews (60 fetched in total)
2022-10-01 14:40:45,226 [INFO] Base - [id:949371011] Fetched 160 reviews (160 fetched in total)
2022-10-01 14:40:52,832 [INFO] Base - [id:949371011] Fetched 240 reviews (240 fetched in total)
2022-10-01 14:41:00,632 [INFO] Base - [id:949371011] Fetched 340 reviews (340 fetched in total)
2022-10-01 14:41:08,627 [INFO] Base - [id:949371011] Fetched 440 reviews (440 fetched in total)
2022-10-01 14:41:16,796 [INFO] Base - [id:949371011] Fetched 520 reviews (520 fetched in total)
2022-10-01 14:41:23,974 [INFO] Base - [id:949371011] Fetched 600 reviews (600 fetched in total)
2022-10-01 14:41:30,641 [INFO] Base - [id:949371011] Fetched 700 reviews (700 fetched in total)
2022-10-01 14:41:38,632 [INFO] Ba

vib1: (1248, 7)


2022-10-01 14:42:51,622 [INFO] Base - Initialised: AppStore('vn', 'myvib-2-0', 1626624790)
2022-10-01 14:42:51,622 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/vn/app/myvib-2-0/id1626624790
2022-10-01 14:42:57,628 [INFO] Base - [id:1626624790] Fetched 100 reviews (100 fetched in total)
2022-10-01 14:43:04,175 [INFO] Base - [id:1626624790] Fetched 200 reviews (200 fetched in total)
2022-10-01 14:43:11,048 [INFO] Base - [id:1626624790] Fetched 320 reviews (320 fetched in total)
2022-10-01 14:43:15,492 [INFO] Base - [id:1626624790] Fetched 400 reviews (400 fetched in total)


vib2: (400, 7)
(1648, 8)


C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_19320\2117567824.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ios_used.rename(columns = {'userName':'username'}, inplace=True)
C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_19320\2117567824.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ios_used['date'] = df_ios_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))


In [8]:
df_used = pd.concat([df_android_used, df_ios_used])
df_used['duplicate'] = df_used.duplicated(subset=['username', 'm_y', 'review'], keep='first')
print('BEFORE: df_used.shape = ',df_used.shape[0])
df_used = df_used[df_used['duplicate']==False] #remove duplicate
df_used.reset_index(inplace=True, drop=True)
print('AFTER DUPLICATE PROCESSING: df_used.shape = ',df_used.shape[0])

# process for date column
df_used['date'] = df_used['date'].apply(lambda x: ('20'+x) if (len(x)==17) else x ) # add '20' before datetime string for right format yyyy-mm-dd 
df_used['date'] = df_used['date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

df_used.to_csv(PRED_ROOT+'/2_dataApp_combine.csv',index=False,sep='\t')

BEFORE: df_used.shape =  8030
AFTER DUPLICATE PROCESSING: df_used.shape =  7779


In [9]:
# df_used = pd.read_csv(PRED_ROOT+'/2_dataApp_combine.csv',index_col=False,sep='\t')
# df_used['date'] = pd.to_datetime(df_used['date'])

__________________________________________________________________________________________________________________________
### Data Processing and Prediction

Vietnamese process --> English translation --> Clean text --> extract TFIDF + VADER + NEGWORD feature --> PREDICTION

In [16]:
# df_used['ymdhms'] = pd.to_datetime(df_used['date']).dt.strftime("%Y_%m_%d_%H_%M_%S")
df_used['date'] = pd.to_datetime(df_used['date'])

# Get the piece of data the you would like to predict.
df_new = df_used[df_used['date']>=GET_SENTIMENT_OF_REVIEW_FROM_DATE].reset_index(drop=True)
df_new = df_new[df_new['date']<=GET_SENTIMENT_OF_REVIEW_TO_DATE].reset_index(drop=True)

# RUN SENTIMENT PREDITION
df_pred_result = sentiment_prediction(df_new, tfidf_convertor='./model/TFIDF_VECTORIZER_app.pk', \
    sentiment_model='./model/SENTIMENT_MODEL_svc_app_nbank.sav', pred_out_file=PRED_ROOT+'/OUT_sentiment_prediction.csv')
df_pred_result[['review_adj','review_en','review_clean','vader_compound','sentiment']]

There are 157 new reviews


,review_adj,review_en,review_clean,vader_compound,sentiment
0,muốn mơ tai khoan vib luôn cho căn cước mô đượ...,want to have a vib earphone for registered tis...,want earphone register tissue id,0,0
1,không mở được trên redmi,can't open on redmi,can not open,0,1
2,không có đăng ký chỉ có đăng nhập thôi hả,No registration only login,registration login,0,1
3,rất hay,very nice,nice,0,0
4,vô trang vay trong,no loan page in,loan page,0,0
...,...,...,...,...,...
152,app mới như không được có mỗi cái tính năng ch...,The new app doesn't have the money transfer fe...,new does not money transfer feature many step ...,0,1
153,đưa ra nhiều chương trình ưu đãi miễn phí thườ...,offer many annual free incentive programs with...,offer many annual free incentive program uncle...,0,1
154,lỗi quá nhiều,too many errors,many error,1,1
155,một cái app ngân hàng ngu chưa từng thấy,a stupid banking app ever,stupid banking ever,1,1


In [17]:
avg_sentiment_score = df_pred_result['sentiment'].sum()/df_pred_result.shape[0]
sentiment_score = -(avg_sentiment_score-0.5)*2
print(f"Neg/Pos ratio: {df_pred_result['sentiment'].sum()}/{df_pred_result.shape[0]-df_pred_result['sentiment'].sum()}")
print(f"Sentiment Score: {sentiment_score:.5f}")

Neg/Pos ratio: 100/54
Sentiment Score: -0.29870


In [26]:
pd.options.display.max_colwidth = 100
df_pred_result[df_pred_result['sentiment']==1]['review_adj'][60:100]

102    dịch vụ khách hàng quá kém cả ngày gọi chục cuộc mới nghe máy bực mình nhân viên mỗi người tư vấ...
104                                                                                               lỗi suốt
105    là ngân hàng lớn mà gọi điện thoại nhờ hỗ trợ không bao giờ bắt máy trong khi đó mình toàn gọi t...
106                                                           dm app như tồi gọi tổng đài lúc nào cũng bận
107                                                                                    cứ bị lỗi gián đoạn
110                                                                               ngâm hồ sơ khách quá lâu
113             app ngu không chịu được lỗi liên tục công ty trả lương xong không chuyển đi nổi vì quá lỗi
114                                                                                           lỗi liên tục
117                                     đăng ký smart otp mà toàn báo hệ thống đang bận không đăng ký được
118    ứng dụng có như không có chụp 

### END
______________________________________________________________________________________________________________________________________________________________________________________

In [ ]:
sys.exit()

SystemExit: 

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#### DEMO

In [59]:
get_review_in = ['stb']

rw_android_src_dict = {'vib1':['com.vn.vib.mobileapp','vi','vn'],
                       'vib2':['com.vib.myvib2','vi','vn'],
                       'tcb':['vn.com.techcombank.bb.app','vi','vn'],
                       'tpb':['com.tpb.mb.gprsandroid','vi','vn'],
                       'bid':['vn.com.bidv.ibank','vi','vn'],
                       'vcb':['com.VCB','vi','vn'],
                       'mb':['com.mbcorp','vi','vn'],
                       'stb':['com.sacombank.ewallet','vi','vn']
                       }
#----------------------------------------------------------------------------------------------------------
# get Android review
column = ['reviewId','userName','userImage','content','score','thumbsUpCount','reviewCreatedVersion','at','replyContent','repliedAt','bankapp']
df_android_rw = pd.DataFrame(columns = column)

for bank_name in get_review_in:
    Andrw_df = get_rwdata_android(rw_android_src_dict, bank_name, save_path=PRED_ROOT)   
    Andrw_df['bankapp'] = bank_name
    df_android_rw = pd.concat([df_android_rw, Andrw_df])
    time.sleep(30)
    
df_android_rw.reset_index(drop=True, inplace=True)
df_android_rw.to_csv(PRED_ROOT+'/1_AndRaw_'+str(today)+'all.csv', index=False)

print(df_android_rw.shape)
df_android_rw.head(1)

#----------------------------------------------------------------------------------------------------------
df_android_rw['month'] = pd.DatetimeIndex(df_android_rw['at']).month
df_android_rw['year'] = pd.DatetimeIndex(df_android_rw['at']).year
df_android_rw['m_y'] = df_android_rw['month'].astype(str)+df_android_rw['year'].astype(str)

#JUST RAW
df_android_used = df_android_rw[['userName','content','at','m_y','score']]
df_android_used.rename(columns = {'userName':'username','content':'review','at':'date','score':'rating'}, inplace=True)

# Convert timetamps to string
df_android_used['date'] = df_android_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
print(df_android_used.shape)

#----------------------------------------------------------------------------
df_used = df_android_used
df_used['ymdhms'] = pd.to_datetime(df_used['date']).dt.strftime("%Y_%m_%d_%H_%M_%S")
df_used['date'] = pd.to_datetime(df_used['date'])

# Get the piece of data the you would like to predict.
df_new = df_used[df_used['date']>=GET_SENTIMENT_OF_REVIEW_FROM_DATE].reset_index(drop=True)

# RUN SENTIMENT PREDITION
df_pred_result = sentiment_prediction(df_new, tfidf_convertor='./model/TFIDF_VECTORIZER_app.pk', sentiment_model='./model/SENTIMENT_MODEL_svc_app_nbank.sav', pred_out_file=PRED_ROOT+'/OUT_sentiment_prediction.csv')
df_pred_result[['review_adj','review_en','review_clean','vader_compound','sentiment']]

stb: (13959, 10)
(13959, 11)
(13959, 5)


C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_25672\3261132450.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_android_used.rename(columns = {'userName':'username','content':'review','at':'date','score':'rating'}, inplace=True)
C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_25672\3261132450.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_android_used['date'] = df_android_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_25672\3261132450.py:44: SettingWithCopyWarning: 
A 

There are 355 new reviews


,review_adj,review_en,review_clean,vader_compound,sentiment
1,okay,okay,okay,0,0
2,keu kich hoat the được tang kich hoat roi không thay gì lun,The operation will be increased and it will not change anything,operation increase not change anything,0,1
3,chuyển tiền thất bại sao vẫn trừ tiền vậy kêu đợi ngày xử lý vẫn không thấy hoàn tiền lại là sao còn cập nhật mới kg sài vân tay được là sao vậy,"The money transfer failed, why is it still deducting the money so I'm waiting for the processing date but I still don't see the refund, why can't I still update the fingerprint?",money transfer fail still deduct money i am wait processing date still do not see refund can not still update fingerprint,1,1
4,không thể mở tài khoản online,Can't open an online account,can not open online account,0,1
5,cho sao nha mua cac mat tiền kho thai car,Why don't you buy some money for Thai car,do not buy money car,0,1
...,...,...,...,...,...
350,rất tốt,very good,good,0,0
351,tốt,good,good,0,0
352,ứng dụng thông mình và hữu ích nè,Smart and useful app,smart useful,0,0
353,giao diện đơn giản dễ thao tác,Simple interface easy to operate,simple interface easy operate,0,0


In [60]:
avg_sentiment_score = df_pred_result['sentiment'].sum()/df_pred_result.shape[0]
sentiment_score = -(avg_sentiment_score-0.5)*2
print(f"Neg/Pos ratio: {df_pred_result['sentiment'].sum()}/{df_pred_result.shape[0]-df_pred_result['sentiment'].sum()}")
print(f"Sentiment Score: {sentiment_score:.5f}")

Neg/Pos ratio: 234/113
Sentiment Score: -0.34870


In [73]:
pd.options.display.max_colwidth = 3000
df_pred_result[df_pred_result['sentiment']==1]['review_adj'][25:50]

53                                                                                                                                                                                                                                           không cho xác thực sự quá tệ
54                                                                                                                                                           app gì suốt ngày toàn gửi otp làm cái không gì không biết vào nạp card hay chuyển khoản gì cũng không được 🤔
55                                                                                                                                                                                         lỗi không vào được ứng dụng bằng mạng báo lỗi hơn tháng rồi vẫn chưa khắc phục
58                                                                                                                                                                                                        

### [PHUONG] SENTIMENT PREDICTION for "Interal Rating" data 


In [ ]:
import cx_Oracle as ora

# Query data from DEC
dsnStr = ora.makedsn(host = '10.60.54.12', port = '1521', service_name = 'DEC_DATA')
con = ora.connect(user = 'ADM42244NHANNT', password = 'Vibhn321#!', dsn = dsnStr)
c = con.cursor()
result = c.execute('select * from dec_bu.u_raw_internal_rating_info').fetchall()
df_ir = pd.DataFrame(result)
df_ir.columns = [x[0] for x in c.description]

# fill Nan Value
df_ir['RATING_COMMENT'] = df_ir['RATING_COMMENT'].apply(lambda x: 0 if (x=='nan') else x)

# Get comment having content
df_w = df_ir[df_ir['RATING_COMMENT']!=0]
#df_w = df_w[['USER_NAME','RATING_COMMENT','RATING_POINT']]
df_w = df_w.rename(columns={'USER_NAME':'username','RATING_COMMENT':'review','RATING_POINT':'rating'})
print(df_w.shape[0])

# Run Sentiment Prediction
df_pred_result = sentiment_prediction(df_w, tfidf_convertor='./model/TFIDF_VECTORIZER_app.pk', sentiment_model='./model/SENTIMENT_MODEL_mnb_app.sav', pred_out_file='./internal_rating_sentiment.csv')
df_pred_result[['review','review_en','review_clean','vader_compound','sentiment']]